## Setups

In [8]:
import pandas as pd
import pybroker as pyb
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance, StrategyConfig

from datetime import datetime, timedelta
from numba import njit
import numpy as np

In [9]:
config = StrategyConfig(max_long_positions=5)
pyb.param('target_size', 1 / config.max_long_positions)
pyb.param('rank_threshold', 6)
pyb.enable_data_source_cache('yfinance')

## Indicators

In [11]:
import talib as ta

def ew13612_fnc(data):
    roc12=ta.ROC(data.adj_close, timeperiod=12)
    roc6=ta.ROC(data.adj_close, timeperiod=6)
    roc3=ta.ROC(data.adj_close, timeperiod=3)
    roc1=ta.ROC(data.adj_close, timeperiod=1)
    
    return (roc12 + roc6 + roc3 + roc1)/4

ew13612_idx = pyb.indicator('ew13612_idx', lambda data: ew13612_fnc(data))

def adj_close_fnc(data):
    v_adj_close=data.adj_close
    
    return v_adj_close

adj_close_idx = pyb.indicator('adj_close_idx', lambda data: adj_close_fnc(data))


def pctl_chnl_fnc(data, lookback):

    # @njit  # Enable Numba JIT.
    def vec_pctl_chnl_fnc(data):
        rolling_close = data.rolling(lookback)
        pctl25 = rolling_close.quantile(0.25)
        pctl75 = rolling_close.quantile(0.75)
        closing_price = data.adj_close

        result = np.where(closing_price < pctl25, -1, np.where(closing_price > pctl75, 1, 0))
        return result
    return vec_pctl_chnl_fnc(data)
        
pc252_idx = pyb.indicator('pc252_idx', pctl_chnl_fnc, lookback=200)



## Data

In [163]:


md = pd.read_csv('D:/Documents/Development/Jupyter/pybroker/market_dates.csv', parse_dates=['trade_dt'], index_col='trade_dt')
# md.sort_values(by=['trade_dt'])[-20:]

yfinance = YFinance()

# symbols=['SPY','QQQ','IWM','VGK','EWJ','EEM','VNQ','PDBC','GLD','HYG','LQD','TLT']
symbols=['SPY','DBC','LQD','VNQ']

df = yfinance.query(symbols, start_date='1/1/2015', end_date='8/1/2023')

# df.rename(columns={"date":"trade_dt"}, inplace=True)
# df.set_index('trade_dt', inplace=True)
#[['Symbol']=='EEM']
dom_df = pd.merge(df, md[['month_last_day']], left_on='date', right_on='trade_dt')
eom_df = dom_df[pos_assets_df['month_last_day'] == 'Y'][['date','symbol','adj_close']]
eom_df

Loaded cached bar data.



,date,symbol,adj_close
76,2015-01-30,DBC,16.804605
77,2015-01-30,SPY,171.007416
78,2015-01-30,VNQ,61.751835
79,2015-01-30,LQD,94.310707
152,2015-02-27,DBC,17.548258
...,...,...,...
8551,2023-06-30,LQD,107.423195
8628,2023-07-31,DBC,24.680000
8629,2023-07-31,SPY,457.790009
8630,2023-07-31,VNQ,85.260002


## Percentile Score

In [250]:
pc_score = eom_df.copy()
for lookback in [3,6,9,12]:
    rolling_close = pc_score['adj_close'].rolling(lookback)
    pctl25 = rolling_close.quantile(0.25)
    pctl75 = rolling_close.quantile(0.75)
    close_price = pc_score['adj_close']

    # result = pd.Series(-1, index=close_price.index)
    result = pd.Series(-1, index=close_price.index)
    last_result=-1
    for idx, value in close_price.items():
        if close_price[idx] > pctl75[idx]:
            result[idx]=1
            last_result=1
        elif (close_price[idx] > pctl25[idx]) & (last_result==1):
            result[idx]=1
            last_result=1
        
        pc_column = f"pc_{lookback}"
        pc_score[pc_column] = result

pc_score.set_index('date', inplace=True)
pc_score

,symbol,adj_close,pc_3,pc_6,pc_9,pc_12
date,,,,,,
2015-01-30,DBC,16.804605,-1,-1,-1,-1
2015-01-30,SPY,171.007416,-1,-1,-1,-1
2015-01-30,VNQ,61.751835,-1,-1,-1,-1
2015-01-30,LQD,94.310707,-1,-1,-1,-1
2015-02-27,DBC,17.548258,-1,-1,-1,-1
...,...,...,...,...,...,...
2023-06-30,LQD,107.423195,1,1,1,1
2023-07-31,DBC,24.680000,-1,-1,-1,-1
2023-07-31,SPY,457.790009,1,1,1,1


## Percentile Channel Score

In [261]:
pc_chan_score = pc_score.copy()
def sum_pc(row):
    pc_columns = ['pc_3', 'pc_6', 'pc_9', 'pc_12']
    return row[list(pc_columns)].sum()/4

pc_chan_score['pc_chan_score'] = pc_chan_score.apply(sum_pc, axis=1)
pc_chan_score['pc_chan_score_abs'] = pc_chan_score.apply(sum_pc, axis=1).abs()
pc_chan_score[-6:]

,symbol,adj_close,pc_3,pc_6,pc_9,pc_12,pc_chan_score,pc_chan_score_abs
date,,,,,,,,
2023-06-30,VNQ,83.559998,1,1,1,1,1.0,1.0
2023-06-30,LQD,107.423195,1,1,1,1,1.0,1.0
2023-07-31,DBC,24.680000,-1,-1,-1,-1,-1.0,1.0
2023-07-31,SPY,457.790009,1,1,1,1,1.0,1.0
2023-07-31,VNQ,85.260002,1,1,1,1,1.0,1.0
2023-07-31,LQD,107.480995,1,1,1,1,1.0,1.0


## Volatility

In [257]:
pc_vol = df[['date','symbol','adj_close']].copy() # yfinance.query(symbols, start_date='1/1/2015', end_date='8/1/2023')
# pc_vol=pc_vol[['date','symbol','adj_close']].copy()

pc_vol['date'] = pd.to_datetime(pc_vol['date'])

# Sort the DataFrame by 'symbol' and 'date' to ensure proper order for rolling calculation
pc_vol.sort_values(by=['symbol', 'date'], inplace=True)

# Set 'date' as the index for the DataFrame
pc_vol.set_index('date', inplace=True)

# Calculate the rolling 20-day volatility for each symbol
pc_vol['vol20'] = pc_vol.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).std())
# pc_vol['vol20_abs'] = pc_vol.groupby('symbol')['adj_close'].transform(lambda x: x.rolling(window=20).std()).abs()

pc_vol[-4:]

,symbol,adj_close,vol20
date,,,
2023-07-26,VNQ,86.660004,1.311900
2023-07-27,VNQ,84.779999,1.125717
2023-07-28,VNQ,84.709999,1.019812
2023-07-31,VNQ,85.260002,0.935763


##  Channel Score Plus Volatility 

In [262]:
pc_vol.reset_index(inplace=True)

# Merge based on 'date' and 'symbol'
pc_chan_score_vol = pd.merge(
    pc_chan_score,
    pc_vol[['date','symbol','vol20']],
    on=['date', 'symbol'],
    how='left'
)

pc_chan_score_vol

,date,symbol,adj_close,pc_3,pc_6,pc_9,pc_12,pc_chan_score,pc_chan_score_abs,vol20
0,2015-01-30,DBC,16.804605,-1,-1,-1,-1,-1.0,1.0,0.297524
1,2015-01-30,SPY,171.007416,-1,-1,-1,-1,-1.0,1.0,1.867482
2,2015-01-30,VNQ,61.751835,-1,-1,-1,-1,-1.0,1.0,1.360318
3,2015-01-30,LQD,94.310707,-1,-1,-1,-1,-1.0,1.0,0.756146
4,2015-02-27,DBC,17.548258,-1,-1,-1,-1,-1.0,1.0,0.215119
...,...,...,...,...,...,...,...,...,...,...
407,2023-06-30,LQD,107.423195,1,1,1,1,1.0,1.0,0.524190
408,2023-07-31,DBC,24.680000,-1,-1,-1,-1,-1.0,1.0,0.665887
409,2023-07-31,SPY,457.790009,1,1,1,1,1.0,1.0,6.262987
410,2023-07-31,VNQ,85.260002,1,1,1,1,1.0,1.0,0.935763


## Volatility Adjusted Channel Score

In [269]:
pc_vol_adj_score=pc_chan_score_vol.copy()
pc_vol_adj_score['chan_score_inv_vol'] = pc_chan_score_vol['pc_chan_score'] * (1 / pc_chan_score_vol['vol20'])
pc_vol_adj_score['chan_score_inv_vol_abs'] = pc_vol_adj_score['chan_score_inv_vol'].abs()
pc_vol_adj_score

,date,symbol,adj_close,pc_3,pc_6,pc_9,pc_12,pc_chan_score,pc_chan_score_abs,vol20,chan_score_inv_vol,chan_score_inv_vol_abs
0,2015-01-30,DBC,16.804605,-1,-1,-1,-1,-1.0,1.0,0.297524,-3.361077,3.361077
1,2015-01-30,SPY,171.007416,-1,-1,-1,-1,-1.0,1.0,1.867482,-0.535481,0.535481
2,2015-01-30,VNQ,61.751835,-1,-1,-1,-1,-1.0,1.0,1.360318,-0.735122,0.735122
3,2015-01-30,LQD,94.310707,-1,-1,-1,-1,-1.0,1.0,0.756146,-1.322496,1.322496
4,2015-02-27,DBC,17.548258,-1,-1,-1,-1,-1.0,1.0,0.215119,-4.648586,4.648586
...,...,...,...,...,...,...,...,...,...,...,...,...
407,2023-06-30,LQD,107.423195,1,1,1,1,1.0,1.0,0.524190,1.907706,1.907706
408,2023-07-31,DBC,24.680000,-1,-1,-1,-1,-1.0,1.0,0.665887,-1.501757,1.501757
409,2023-07-31,SPY,457.790009,1,1,1,1,1.0,1.0,6.262987,0.159668,0.159668
410,2023-07-31,VNQ,85.260002,1,1,1,1,1.0,1.0,0.935763,1.068647,1.068647


## Weightings

In [272]:
vol_adj_date_sum = pc_vol_adj_score.groupby('date')['chan_score_inv_vol_abs'].sum()
pc_score_vol_adj_dt_wght = pc_vol_adj_score.copy()

for date, value in vol_adj_date_sum.items():
    pc_score_vol_adj_dt_wght.loc[pc_score_vol_adj_dt_wght['date'] == date, 'vol_adj_sum_date'] = value

pc_score_vol_adj_dt_wght

,date,symbol,adj_close,pc_3,pc_6,pc_9,pc_12,pc_chan_score,pc_chan_score_abs,vol20,chan_score_inv_vol,chan_score_inv_vol_abs,vol_adj_sum_date
0,2015-01-30,DBC,16.804605,-1,-1,-1,-1,-1.0,1.0,0.297524,-3.361077,3.361077,5.954176
1,2015-01-30,SPY,171.007416,-1,-1,-1,-1,-1.0,1.0,1.867482,-0.535481,0.535481,5.954176
2,2015-01-30,VNQ,61.751835,-1,-1,-1,-1,-1.0,1.0,1.360318,-0.735122,0.735122,5.954176
3,2015-01-30,LQD,94.310707,-1,-1,-1,-1,-1.0,1.0,0.756146,-1.322496,1.322496,5.954176
4,2015-02-27,DBC,17.548258,-1,-1,-1,-1,-1.0,1.0,0.215119,-4.648586,4.648586,5.178171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,2023-06-30,LQD,107.423195,1,1,1,1,1.0,1.0,0.524190,1.907706,1.907706,6.014790
408,2023-07-31,DBC,24.680000,-1,-1,-1,-1,-1.0,1.0,0.665887,-1.501757,1.501757,3.917919
409,2023-07-31,SPY,457.790009,1,1,1,1,1.0,1.0,6.262987,0.159668,0.159668,3.917919
410,2023-07-31,VNQ,85.260002,1,1,1,1,1.0,1.0,0.935763,1.068647,1.068647,3.917919


In [286]:

pc_pct_alloc=pc_score_vol_adj_dt_wght.copy()
# pc_pct_alloc['pct_alloc'] = pc_pct_alloc['pc_score_vol_adj'] / pc_pct_alloc['score_vol_adj_date_wght']
pc_pct_alloc['pct_alloc'] = (pc_pct_alloc['chan_score_inv_vol_abs'] / pc_pct_alloc['vol_adj_sum_date']).apply(lambda x: '{:.2f}%'.format(x * 100))


pc_pct_alloc

,date,symbol,adj_close,pc_3,pc_6,pc_9,pc_12,pc_chan_score,pc_chan_score_abs,vol20,chan_score_inv_vol,chan_score_inv_vol_abs,vol_adj_sum_date,pct_alloc
0,2015-01-30,DBC,16.804605,-1,-1,-1,-1,-1.0,1.0,0.297524,-3.361077,3.361077,5.954176,56.45%
1,2015-01-30,SPY,171.007416,-1,-1,-1,-1,-1.0,1.0,1.867482,-0.535481,0.535481,5.954176,8.99%
2,2015-01-30,VNQ,61.751835,-1,-1,-1,-1,-1.0,1.0,1.360318,-0.735122,0.735122,5.954176,12.35%
3,2015-01-30,LQD,94.310707,-1,-1,-1,-1,-1.0,1.0,0.756146,-1.322496,1.322496,5.954176,22.21%
4,2015-02-27,DBC,17.548258,-1,-1,-1,-1,-1.0,1.0,0.215119,-4.648586,4.648586,5.178171,89.77%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,2023-06-30,LQD,107.423195,1,1,1,1,1.0,1.0,0.524190,1.907706,1.907706,6.014790,31.72%
408,2023-07-31,DBC,24.680000,-1,-1,-1,-1,-1.0,1.0,0.665887,-1.501757,1.501757,3.917919,38.33%
409,2023-07-31,SPY,457.790009,1,1,1,1,1.0,1.0,6.262987,0.159668,0.159668,3.917919,4.08%
410,2023-07-31,VNQ,85.260002,1,1,1,1,1.0,1.0,0.935763,1.068647,1.068647,3.917919,27.28%



# GPM Source




In [801]:
pos_assets_df = pd.merge(df, md[['month_last_day']], left_on='date', right_on='trade_dt')
eom_df = pos_assets_df[pos_assets_df['month_last_day'] == 'Y']

ticker_df = eom_df[eom_df['symbol']=='SPY']

mr_srs=adj_close_idx(ticker_df)
ri_srs=ew13612_idx(ticker_df)

mc_df = pd.DataFrame({'SPY': mc_srs})
ri_df = pd.DataFrame({'SPY': ri_srs})

# eom_df[-14:]
ri_df
mc_df

,SPY
2015-01-30,114.027824
2015-01-30,114.027771
2015-02-27,107.028336
2015-02-27,107.028336
2015-03-31,108.197327
...,...
2023-05-31,102.439598
2023-05-31,102.439598
2023-06-30,102.662003
2023-06-30,102.662003


In [802]:
for each_symbol in symbols:
    for lookback_day in [252, 180, 120, 60]:

    ticker_df = eom_df[eom_df['symbol']==each_symbol]
    
    pc_srs=percent_(ticker_df)  # Monthly returns
    ri_srs=ew13612_idx(ticker_df)  # Return Indicator
    
    mc_df[each_symbol] = pd.Series(mc_srs).reindex(mc_df.index)    
    ri_df[each_symbol] = pd.Series(ri_srs).reindex(ri_df.index)    

# ri_df.loc[:, 'SPY'][-12:]
# ri_df
# mc_df # Monthly Closes

# pd.Series(mr_srs).reindex(mr_df.index)

mc_df

,SPY,QQQ,IWM,VGK,EWJ,EEM,VNQ,PDBC,GLD,HYG,LQD,TLT
2015-01-30,171.007385,94.492523,103.350021,39.732143,40.268532,32.641945,61.751839,10.210998,123.449997,58.079361,94.634521,114.027824
2015-01-30,171.007416,94.492523,103.349998,39.732151,40.268536,32.641941,61.751842,10.210998,123.449997,58.079403,94.634514,114.027771
2015-02-27,180.618805,101.315437,109.495628,42.153183,43.282547,34.080803,59.482960,10.614596,116.160004,59.375187,93.290314,107.028336
2015-02-27,180.618774,101.315445,109.495613,42.153179,43.282543,34.080807,59.482964,10.614596,116.160004,59.375156,93.290314,107.028336
2015-03-31,177.782074,98.925400,111.433685,41.125500,43.913372,33.570507,60.513161,9.963796,113.660004,58.812458,93.472305,108.197327
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31,416.303619,347.513306,173.290115,59.174145,58.934296,37.893135,79.106728,13.230000,182.320007,72.965096,106.925797,102.439598
2023-05-31,416.303619,347.513306,173.290115,59.174145,58.934296,37.893135,79.106728,13.230000,182.320007,72.965103,106.925797,102.439598
2023-06-30,443.279999,369.420013,187.270004,61.700001,61.900002,39.560001,83.559998,13.605000,178.270004,74.264000,107.792000,102.662003
2023-06-30,443.279999,369.420013,187.270004,61.700001,61.900002,39.560001,83.559998,13.605000,178.270004,74.264000,107.792000,102.662003


In [803]:
mr_df = mc_df.pct_change()    #percentage_change
#percentage_change[symbols].loc['2023-04-28'].mean()
# percentage_change[['GLD']][-12:]
mr_df

,SPY,QQQ,IWM,VGK,EWJ,EEM,VNQ,PDBC,GLD,HYG,LQD,TLT
2015-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-30,1.784577e-07,0.000000e+00,-2.214628e-07,1.920207e-07,9.473147e-08,-1.168649e-07,6.177463e-08,0.000000,0.000000,7.224885e-07,-8.061957e-08,-4.683573e-07
2015-02-27,5.620452e-02,7.220586e-02,5.946425e-02,6.093383e-02,7.484780e-02,4.408016e-02,-3.674194e-02,0.039526,-0.059052,2.231056e-02,-1.420412e-02,-6.138360e-02
2015-02-27,-1.689612e-07,7.530338e-08,-1.393552e-07,-9.049607e-08,-8.813477e-08,1.119310e-07,6.413093e-08,0.000000,0.000000,-5.139786e-07,0.000000e+00,0.000000e+00
2015-03-31,-1.570546e-02,-2.359014e-02,1.770000e-02,-2.437964e-02,1.457467e-02,-1.497323e-02,1.731920e-02,-0.061312,-0.021522,-9.477000e-03,1.950809e-03,1.092226e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31,4.616256e-03,7.883807e-02,-8.162042e-03,-5.120531e-02,8.497629e-03,-2.402250e-02,-3.961585e-02,-0.065018,-0.013420,-1.231447e-02,-1.768926e-02,-3.015296e-02
2023-05-31,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,1.045622e-07,0.000000e+00,0.000000e+00
2023-06-30,6.479977e-02,6.303847e-02,8.067332e-02,4.268513e-02,5.032224e-02,4.398861e-02,5.629445e-02,0.028345,-0.022214,1.780162e-02,8.100976e-03,2.171079e-03
2023-06-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00


In [750]:
equal_weight_return=percentage_change.mean(axis=1)
equal_weight_return[-12:]

2022-07-29    0.050725
2022-08-31   -0.040255
2022-09-30   -0.083552
2022-10-31    0.029594
2022-11-30    0.072791
2022-12-30   -0.032135
2023-01-31    0.072279
2023-02-28   -0.037609
2023-03-31    0.029426
2023-04-28    0.004523
2023-05-31   -0.014060
2023-06-30    0.036428
dtype: float64

In [756]:
ci_df = percentage_change.rolling(window=12).corr(equal_weight_return)
ci_df

,SPY,QQQ,IWM,VGK,EWJ,EEM,VNQ,DBC,GLD,HYG,LQD,TLT
2015-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28,0.937577,0.912326,0.879118,0.955225,0.970661,0.843418,0.926815,0.432787,0.619499,0.898326,0.920367,0.784663
2023-03-31,0.943347,0.910534,0.816919,0.954009,0.983953,0.863933,0.922850,0.453963,0.619831,0.912528,0.944404,0.822002
2023-04-28,0.937256,0.906274,0.789124,0.947242,0.982370,0.855572,0.942316,0.704292,0.610407,0.903184,0.942212,0.800246
2023-05-31,0.934320,0.859363,0.788594,0.937524,0.979186,0.856900,0.953759,0.762549,0.634723,0.908976,0.948605,0.805414


In [759]:
# zi_df = ri_df.mul(1 - rolling_correlations)
# ri_df
zi_df=ri_df.mul(1 - ci_df)
zi_df

,SPY,QQQ,IWM,VGK,EWJ,EEM,VNQ,DBC,GLD,HYG,LQD,TLT
2015-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28,-0.179389,-0.356755,-0.102119,0.234139,-0.070511,-1.063191,-0.475273,-2.497148,-0.054632,-0.134852,-0.369339,-2.065830
2023-03-31,0.268108,0.893789,-0.217889,0.548579,0.118498,0.397049,-0.280183,-1.670854,3.402759,0.253552,0.157273,0.008895
2023-04-28,0.244384,0.713060,-0.913837,0.657376,0.104371,0.162491,-0.302253,-1.890568,2.919760,0.180169,0.178537,0.218643
2023-05-31,0.206508,2.090131,-1.021474,0.119285,0.092448,-0.454263,-0.375982,-2.941699,2.118703,-0.021319,0.002100,-0.409836


In [807]:
gpm_bkup_df=zi_df.copy()
gpm_bkup_df.reset_index(inplace=True)
gpm_piv_df =gpm_bkup_df.melt(id_vars='index', var_name='ticker',value_name='mom')
gpm_piv_df.rename(columns={'index': 'date'}, inplace=True)
gpm_piv_df.sort_values(by=['date', 'mom'], ascending=[True, False])[-60:]

,date,ticker,mom
403,2023-02-28,VGK,0.234139
913,2023-02-28,GLD,-0.054632
505,2023-02-28,EWJ,-0.070511
301,2023-02-28,IWM,-0.102119
1015,2023-02-28,HYG,-0.134852
97,2023-02-28,SPY,-0.179389
199,2023-02-28,QQQ,-0.356755
1117,2023-02-28,LQD,-0.369339
709,2023-02-28,VNQ,-0.475273
607,2023-02-28,EEM,-1.063191


In [770]:
gpm_df=zi_df.copy()
gpm_df.reset_index(inplace=True)
gpm_piv_df = gpm_df.melt(id_vars='index', var_name='ticker',value_name='mom')
gpm_piv_df.rename(columns={'index': 'date'}, inplace=True)
gpm_piv_df.sort_values(by=['date', 'mom'], ascending=[True, False])[-73:-13]

,date,ticker,mom
1217,2022-12-30,TLT,-3.464216
912,2023-01-31,GLD,4.631220
810,2023-01-31,DBC,1.243283
606,2023-01-31,EEM,0.977468
402,2023-01-31,VGK,0.511877
300,2023-01-31,IWM,0.432289
504,2023-01-31,EWJ,0.174112
1014,2023-01-31,HYG,0.103853
708,2023-01-31,VNQ,0.096335
1116,2023-01-31,LQD,0.074194


In [773]:
gpm_df=ci_df.copy()
# gpm_df.reset_index(inplace=True)
gpm_piv_df = gpm_db.melt(id_vars='index', var_name='ticker',value_name='mom')
gpm_piv_df.rename(columns={'index': 'date'}, inplace=True)
gpm_piv_df.sort_values(by=['date', 'mom'], ascending=[True, False])[-72:-13]

,date,ticker,mom
912,2023-01-31,GLD,0.026722
1218,2023-01-31,TLT,0.016993
606,2023-01-31,EEM,0.015041
300,2023-01-31,IWM,0.011681
708,2023-01-31,VNQ,0.008135
198,2023-01-31,QQQ,0.007549
810,2023-01-31,DBC,0.005605
402,2023-01-31,VGK,0.005141
1116,2023-01-31,LQD,0.004288
96,2023-01-31,SPY,0.003926


In [544]:
gpm_db=result_risk_df.copy()
gpm_db = gpm_db.reset_index()
gpm_db2 = gpm_db.melt(id_vars='index', var_name='ticker',value_name='mom')a
gpm_db3 = gpm_db2.rename(columns={'index': 'date'})
gpm_db3.sort_values(by=['date', 'mom'], ascending=[True, False])[-36:]
# gpm_db2

,date,ticker,mom
915,2023-04-28,GLD,0.003360
405,2023-04-28,VGK,0.002181
99,2023-04-28,SPY,0.001002
1221,2023-04-28,TLT,0.000676
201,2023-04-28,QQQ,0.000476
1119,2023-04-28,LQD,0.000349
1017,2023-04-28,HYG,0.000196
711,2023-04-28,VNQ,0.000181
507,2023-04-28,EWJ,0.000045
609,2023-04-28,EEM,-0.001208


In [266]:
dbx=prtfl_df.copy()
dbx2 = dbx.reset_index()
dbx3 = dbx2.melt(id_vars='index', var_name='ticker',value_name='mom')
dbx4 = dbx3.rename(columns={'index': 'date'})
dbx4.sort_values(by=['date', 'mom'], ascending=[True, False])[-42:]

,date,ticker,mom
405,2023-04-28,VGK,0.142409
915,2023-04-28,GLD,0.092934
201,2023-04-28,QQQ,0.092402
507,2023-04-28,EWJ,0.068406
99,2023-04-28,SPY,0.054355
1119,2023-04-28,LQD,0.029113
1017,2023-04-28,HYG,0.028050
1323,2023-04-28,IEF,0.015117
1425,2023-04-28,SHY,0.012484
609,2023-04-28,EEM,0.009759


In [224]:
dbx

,level_0,index,SPY,QQQ,IWM,VGK,EWJ,EEM,VNQ,DBC,GLD,HYG,LQD,TLT,IEF,SHY
0,0,2015-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2015-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2015-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2015-05-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,2023-02-28,-0.008380,-0.022629,0.023186,0.076887,0.007942,-0.034824,-0.058669,-0.066755,0.008227,0.000105,-0.027149,-0.082822,-0.041132,-0.006987
98,98,2023-03-31,0.032104,0.076445,-0.017634,0.102148,0.062654,0.008332,-0.067154,-0.067578,0.085639,0.023578,0.018079,-0.018099,0.002741,0.010351
99,99,2023-04-28,0.054355,0.092402,-0.024179,0.142409,0.068406,0.009759,-0.047187,-0.068049,0.092934,0.028050,0.029113,0.000869,0.015117,0.012484
100,100,2023-05-31,0.056602,0.167235,-0.028719,0.078170,0.072406,-0.008194,-0.068750,-0.113442,0.075620,0.015302,0.011461,-0.020050,0.002326,0.008245


In [200]:
df_renamed

,SPY,QQQ,IWM,VGK,EWJ,EEM,VNQ,DBC,GLD,HYG,LQD,TLT,IEF,SHY
Date,,,,,,,,,,,,,,
2015-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28,-0.008380,-0.022629,0.023186,0.076887,0.007942,-0.034824,-0.058669,-0.066755,0.008227,0.000105,-0.027149,-0.082822,-0.041132,-0.006987
2023-03-31,0.032104,0.076445,-0.017634,0.102148,0.062654,0.008332,-0.067154,-0.067578,0.085639,0.023578,0.018079,-0.018099,0.002741,0.010351
2023-04-28,0.054355,0.092402,-0.024179,0.142409,0.068406,0.009759,-0.047187,-0.068049,0.092934,0.028050,0.029113,0.000869,0.015117,0.012484


In [189]:
db02 = db01

# db02['positive_assets']

db02['positive_assets'] = db02.apply(lambda row: (row > 0).sum()-1, axis=1)  # The -1 is to not count the index column
db02[-60:]

,SPY,IWM,QQQ,VGK,EWJ,EEM,VNQ,DBC,GLD,HYG,LQD,TLT,IEF,SHY,positive_assets
2018-07-31,0.072748,0.091070,0.110319,0.016610,0.003359,-0.018863,0.038907,0.040507,-0.054236,0.015061,-0.008092,-0.013813,-0.014290,-0.000547,8
2018-08-31,0.091598,0.119291,0.149702,-0.014219,-0.007503,-0.056307,0.062078,0.037088,-0.070235,0.020214,-0.006960,-0.001100,-0.002848,0.002925,7
2018-09-28,0.082716,0.076684,0.124567,-0.015648,0.018353,-0.060265,0.032346,0.058352,-0.068715,0.023161,-0.006796,-0.025242,-0.011387,0.001762,8
2018-10-31,0.001040,-0.043345,0.015852,-0.087714,-0.072686,-0.135757,0.002998,-0.008143,-0.045075,0.002606,-0.024122,-0.049095,-0.011801,0.003306,5
2018-11-30,0.012948,-0.029303,0.004663,-0.087415,-0.061207,-0.086852,0.046110,-0.102902,-0.038695,-0.001352,-0.024174,-0.028562,0.002807,0.006772,5
2018-12-31,-0.073916,-0.138172,-0.082655,-0.122422,-0.124297,-0.110389,-0.032366,-0.132999,0.008716,-0.020875,-0.003986,0.028163,0.029980,0.013297,4
2019-01-31,-0.002057,-0.039840,-0.006225,-0.056882,-0.056345,-0.013153,0.077978,-0.067606,0.037104,0.025316,0.030010,0.033084,0.035364,0.014503,7
2019-02-28,0.029696,0.008753,0.021083,-0.017682,-0.045231,-0.015242,0.076895,-0.037075,0.033204,0.034576,0.027521,0.018327,0.027297,0.013749,10
2019-03-29,0.043252,-0.014767,0.054376,-0.006392,-0.031826,0.003206,0.103379,-0.038521,0.018657,0.042959,0.052926,0.067317,0.048949,0.017897,10
2019-04-30,0.074915,0.014525,0.097113,0.033116,-0.013629,0.032992,0.086310,-0.023979,0.014886,0.047263,0.052929,0.042489,0.039308,0.017608,12


In [181]:
def rank(ctxs: dict[str, ExecContext]):
    scores = {
        symbol: ctx.indicator('sma13m_indicator')[-1]
        for symbol, ctx in ctxs.items()
    }
    sorted_scores = sorted(
        scores.items(),
        key=lambda score: score[1],
        reverse=True
    )

    sorted_scores_trimmed = [(x, y) for x, y in sorted_scores if y > 0]
    
    threshold = pyb.param('rank_threshold')
    # print(sorted_scores_trimmed)
    
    top_scores = sorted_scores_trimmed[:threshold]
    top_symbols = [score[0] for score in top_scores]
    pyb.param('top_symbols', top_symbols)

In [185]:
def rotate(ctx: ExecContext):
    
    day_of_month = md.loc[ctx.dt,'trade_day_nrmlzd']
    shy_signal = ctx.indicator('sma13m_indicator','SHY')[-1]
    ief_signal = ctx.indicator('sma13m_indicator','IEF')[-1] 
    
    asset_signal  = ctx.indicator('sma13m_indicator')[-1]
    
    if day_of_month == 18:
        if ctx.long_pos():
                if ctx.symbol not in pyb.param('top_symbols') or tips_signal <=0:
                    ctx.sell_all_shares()
        else:
            target_size = pyb.param('target_size')
            if ctx.symbol in pyb.param('top_symbols') and tips_signal > 0 and asset_signal > 0:
                ctx.buy_shares = ctx.calc_target_shares(target_size)
                ctx.score = ctx.indicator('sma13m_indicator')[-1]

In [186]:
strategy = Strategy(
    YFinance(),
    start_date='1/1/2015',
    end_date='1/1/2023',
    config=config
)
strategy.set_before_exec(rank)
strategy.add_execution(rotate, ['SPY','QQQ','IWM','VGK','EWJ','EEM','VNQ','DBC','GLD','HYG','LQD','TLT','IEF'], indicators=sma13m_indicator)
result = strategy.backtest(warmup=251)


Backtesting: 2015-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  13 of 13 completed
Loaded bar data: 0:00:01 

Computing indicators...


100% (13 of 13) |########################| Elapsed Time: 0:00:13 Time:  0:00:13



Test split: 2015-01-02 00:00:00 to 2022-12-30 00:00:00


  0% (0 of 2014) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

ValueError: Indicator 'sma13m_indicator' not found for TIP.

In [ ]:
result.trades.sort_values(by=['entry_date','exit_date'])[:20]

In [ ]:
db01.loc['2016-2-29'].sort_values(ascending=False)

In [ ]:
db01.loc['2016-3-31'].sort_values(ascending=False)

In [ ]:
result.metrics_df

In [ ]:
y['entry_year']=y['entry_date'].dt.year

In [ ]:
y.groupby("entry_year").count()

In [ ]:
x=result.orders
x.columns

In [ ]:
result.portfolio

In [ ]:
pctl_chanimport matplotlib.pyplot as plt
chart = plt.subplot2grid((3, 2), (0, 0), rowspan=3, colspan=2)
chart.plot(result.portfolio.index, result.portfolio['market_value'])